In [99]:
# imports
import pandas as pd

## Defining the Turing Machine Structure

Our Turing machine consists of a tape (which can be represented as a list), a head position (an index in the list), and an internal state. Let's define a simple structure to hold these.


In [122]:
#classe da maquina
class TuringMachine:
    #a classe recebe a fita, onde começa a fita, e o estado inicial
    def __init__(self, tape, head_position, initial_state):
        #dados da maquina
        #tape completa
        self.tape = tape
        #posicao do cursor
        self.head_position = head_position
        #estado atual iniciado pelo estado inicial
        self.state = initial_state
    
    #funcao que faz o display da fita
    def display(self):
        print(f"Tape: {self.tape}\nHead Position: {self.head_position}\nState: {self.state}")


## Fetching the Action

We define `get_action` function to fetch the action for the current condition from the program.


In [123]:
#carregar o programa via csv file
program = pd.read_csv('/Users/zuilpirola/Downloads/turing-machine-example-program.csv',delimiter=';')
#print do programa
program

,symbol,state,write,move,new-state
0,0,q0,B,R,q1
1,0,q1,B,R,q0
2,1,q0,B,R,q0
3,1,q1,B,R,q1
4,B,q0,0,L,qf
5,B,q1,1,L,qf
6,B,qf,NaN,NaN,NaN


In [124]:
#funcao que le o programa vindo do csv e gera as instrucoes (acoes)
def get_action(condition, program):
    # Find the action for the given condition in the program
    # criar um dataframe de uma linha a partir 
    action_row = program[(program['state'] == condition[0]) & (program['symbol'] == condition[1])]
    # print("row>>>",action_row)
    if not action_row.empty:
        return action_row.iloc[0]['write'], action_row.iloc[0]['move'], action_row.iloc[0]['new-state']
    else:
        return None


## Updating the Machine

`update_machine` function applies the action to the Turing machine, modifying its tape, head position, and internal state.


In [125]:
def update_machine(machine, action):
    #se uma acao existir
    if action:
        #desemcapsula ela 
        new_symbol, direction, new_state = action
        #atualiza a posicao atual da fita
        machine.tape[machine.head_position] = new_symbol

        #com base na direcao atualiza para esquerda ou direita
        if direction == 'R':
            machine.head_position += 1
        elif direction == 'L':
            machine.head_position -= 1

        #atualiza o novo estado
        machine.state = new_state
        
        # Ensure the tape is long enough
        if machine.head_position >= len(machine.tape):
            machine.tape.append('B') 
        elif machine.head_position < 0:
            machine.tape.insert(0, 'B')
            machine.head_position = 0


## Running the Machine

Finally, `run_machine` function executes the Turing machine, using the `get_action` and `update_machine` functions, until it reaches a final state or there's no action for the current state.


In [126]:
def run_machine(machine, program):

    #correr até chegar em qf
    while True:
        #cria a tupla da acao com base na condição extraída da tabela do programa
        condition = (machine.state, machine.tape[machine.head_position])
        action = get_action(condition, program)

        #funcao que atualiza a maquina com base na acao atual
        update_machine(machine, action)
        
        #condicao de saída
        if action[2] == 'qf':
            break

        #print da situação atual        
        machine.display()


## Execution Example

Let's put all the pieces together and run our Turing machine with an example program and input.


In [127]:
# Input
initial_tape = ['B', 'B', '0', '0', '0', 'B', 'B','B']
head_position = 2
initial_state = 'q0'

# Initialize the machine
machine = TuringMachine(initial_tape, head_position, initial_state)

# Display do status inicial
machine.display()

# print()
run_machine(machine, program)

# Display do status Final
machine.display()


Tape: ['B', 'B', '0', '0', '0', 'B', 'B', 'B']
Head Position: 2
State: q0
Tape: ['B', 'B', 'B', '0', '0', 'B', 'B', 'B']
Head Position: 3
State: q1
Tape: ['B', 'B', 'B', 'B', '0', 'B', 'B', 'B']
Head Position: 4
State: q0
Tape: ['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B']
Head Position: 5
State: q1
Tape: ['B', 'B', 'B', 'B', 'B', '1', 'B', 'B']
Head Position: 4
State: qf
